## Introduction

I once heard about <a href="https://en.wikipedia.org/wiki/Monty_Hall_problem">The Monty Hall Problem</a>, a riddle or brain teaser involving decisions based on probility.  In this scenario, a contestant on a game show is shown three closed doors and told there is a valuable prize behind one and a goat behind the other two.  (Assume the contestant does not value goats.)  They are given the chance to pick a door, and will receive the item that is behind the door.  However, after the contestant picks, the game show host, who knows what is behind each door, opens one of the doors the contestant did not pick to reveal a goat.  Now there is one door open (with a goat) and two remain closed, one of which is the contestant's choice.  The host gives the contestant an opportunity to keep their original pick or change, before all the doors are opened and the contestant gets the prize or a goat.

The question is, as this point, <b>is it better for the contestant to keep the original choice or change choices?  Or does it matter?</b>

Thare is a theoretical answer based on probability calculation.  I thought it would be fun to simulate it in code, to check the theoretical answer.

## Question for the code to answer
At the point of choosing to keep the original choice or change choices, are you more likely to get the prize by switching or keeping the original choice?

## Methodology

I wrote a simple program to simulate the choice many times and add up how many times the contestant got the prize and how many times they got the goat.

In the do_one_turn() function, a random door is chosen to be the prize door, and a random door is chosen, independently to be the original choice.  As in the game show, this has to happen <i>before</i> anything is revealed and a choice is changed.  Now, the door that is neither a choice nor a prize is opened to reveal a goat.

In half of the scenrios, the contestant choosed to switch doors.  In the other half they keep their choice.

In [13]:
from numpy import random
from datetime import datetime

In [14]:
DEFAULT_N = 10000

In [15]:
def run_sim(n=DEFAULT_N):
    print("run_sim", n)
    start = datetime.now()
    num_wins_changed_choice = 0
    for i in range(n):
        num_wins_changed_choice += do_one_turn(True)

    num_wins_kept_choice = 0
    for i in range(n):
        num_wins_kept_choice += do_one_turn(False)

    stop = datetime.now()
    print('completed in ', (stop.timestamp() - start.timestamp()), 'seconds')
    return num_wins_changed_choice / n * 100, num_wins_kept_choice / n * 100

In [16]:
def do_one_turn(change_choice=True):
    door_numbers = [1, 2, 3]
    prize_door = random.choice(door_numbers)
    door_choice = random.choice(door_numbers)

    # Next, start the process to select a door to open.
    doors_to_reveal_from = door_numbers.copy()
    doors_to_reveal_from.remove(door_choice)    # because you don't open the one that was chosen

    # Now remove the prize door from the list of doors to potentially open.
    if prize_door in doors_to_reveal_from:
        # We are checking if it there so we don't get an exception.
        doors_to_reveal_from.remove(prize_door)

    # Now, doors_to_reveal from does not contain the original choice or the prize door.
    # It might be one or two doors, depending on whether the contestant chose the prize door.
    # So open the non prize door.
    revealed_door = random.choice(doors_to_reveal_from)

    # If contestant wants to change their choice.
    if change_choice:
        doors = door_numbers.copy()
        doors.remove(revealed_door)
        doors.remove(door_choice)
        door_choice = doors.pop()

    return door_choice == prize_door

## Results

In [19]:
for i in range(3, 7):
    results = run_sim(10 ** i)
    print(results[0], ' got the prize when switching')
    print(results[1], ' got the prize when staying with the original choice')
    print()

run_sim 1000
completed in  0.04291105270385742 seconds
66.3  got the prize when switching
30.5  got the prize when staying with the original choice

run_sim 10000
completed in  0.3780369758605957 seconds
66.52  got the prize when switching
33.21  got the prize when staying with the original choice

run_sim 100000
completed in  3.677156925201416 seconds
66.63  got the prize when switching
33.501  got the prize when staying with the original choice

run_sim 1000000
completed in  37.94769597053528 seconds
66.63470000000001  got the prize when switching
33.4022  got the prize when staying with the original choice



You can see in the above output that in about 2/3 of the time, when they switched choiced, they ended up winning the prize.  They won the prize in about 1/3 of the time they did not switch.  This is relatively constant throughout a range of how many times you do the simulation.

## Explanation
At the time the host opens a goat door and gives the contestant the chance to keep their original choice or change it, it is tempting to say that there is a 50 / 50 chance of the prize being being either door.  I am tempted by that also.  But you have to go back and look at the entire sequence of events.

At the start of it all, there are three doors.  You don't know what is being each one.  At this time, whatever door you pick has a one in three chance of being the prize door.

When the first goat door is removed in the next step, does that change the fact that, 10 seconds ago, you  had a one in three chance of picking the prize door?  No.  The past has not changed.  Your pick still has a 1/3 chance of being correct.  So the other door, the one that is still closed but was not your pick, must have a 2/3 chance of containing the prize.  

When I originally heard about this on a podcast, the speaker explained it by saying that it is as if, when the first goat door is opened, all it's probablity transfers over to the other door you did not pick.  Whether you prefer to think of it as a transfer or probability or the "what's done is done" way that I explained it, the point is that the theory says that it is in the contestant's best interest to switch their choice*.  And this simulation seems to perfectly agree with the theory.

*This analysis is specific to the Monty Hall Problem and does not necessarily generalize to other areas of life.  That is, in other contexts you may or may not want to go with the newest option that comes along in such areas as stock picking, house buying, who you are married to, or what gender you are.